# CET 324 Authentication System

__Notes__    
Used bcrypt for reasons given by: https://blog.cloudflare.com/keeping-passwords-safe-by-staying-up-to-date/    
Used AES for encryption of token. Create a class with a defined key for the AES encryption.    
Used Asymmetric encryption to encrypt the private shared key for Symmetric encryption

All functions provided. Maybe look at hashing the whole token rather than just the signature. Look at making an admin class with extra privilages (eg. make another user an admin/pw reset/new key for system encryption)

In [85]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import hashlib, io, hmac
from cryptography.fernet import Fernet
import csv
import bcrypt
import maskpass 
from IPython.display import clear_output 
import time
import rsa
from base64 import b64encode, b64decode

In [86]:
class User:
    def __init__(self, username, password, permission):
        self.username = username
        self.password = password
        self.permission = permission
        
    def requestAccessToken(self, system):
        accessToken = AccessToken(self, system)
        return accessToken.generateToken().decode('utf-8')
    
    def userAccountPage(self, system):
        clear_output(wait=True)
        print(self.username + "\'s Account Page")
        print("___________________")
        while 1: 
            print("1. Request token")
            print("0. Logout")
            
            user_command = int(input("Enter your command: "))
            
            #if else statement to carry out user command 
            if user_command == 1:
                new_token = self.requestAccessToken(system)
                print(new_token)
            elif user_command == 0:
                print("Logging Out")
                #give time for user to see log out message
                time.sleep(2)
                clear_output(wait=True)
                break
            else: 
                clear_output(wait=True)
                print("*** Invalid Command ***")
                print("Please Try Again")
                
class AdminUser(User): 
    def userAccountPage(self, system):
        clear_output(wait=True)
        print(self.username + "\'s Admin Account Page")
        print("___________________")
        while 1: 
            print("1. Request token")
            print("2. View all Users")
            print("3. Create New System Key")
            print("0. Logout")
            
            user_command = int(input("Enter your command: "))
            
            #if else statement to carry out user command 
            if user_command == 1:
                new_token = self.requestAccessToken(system)
                print(new_token)
            elif user_command == 2: 
                self.viewUsers(system.systemName)
            elif user_command == 3:
                self.createNewSystemKey(system)
            elif user_command == 0:
                print("Logging Out")
                time.sleep(2)
                clear_output(wait=True)
                break
            else: 
                clear_output(wait=True)
                print("*** Invalid Command ***")
                print("Please Try Again")
                
                
    def createNewSystemKey(self, system):
        system.createSystemKey()
        print("New Key created, all access tokens are now invalid")
               
    #method to view list of all users 
    def viewUsers(self, systemName):
        with open(systemName + 'users.csv', newline='') as userscsv:
            fieldnames = ['username', 'password', 'permission']
            reader = csv.DictReader(userscsv, fieldnames=fieldnames)
            for row in reader:
                print("", row['username'],"Permission: ", row['permission'])            

In [87]:
class AccessToken:
    #declare a splitter (symbols which will be used to seperate data within the token)
    SPLIT = "!£$"
    
    def __init__(self, user, system):
        self.user = user
        self.system = system
        self.date = date.today()
        self.token = ''
        self.encrypt = Encryption(self.system)
    
    def calcExpiryDate(self):
        #add 6 months to current date to calculate expiry date 
        return self.date + relativedelta(months=6)
    
    def generateToken(self): 
        splitter = self.SPLIT
        #generate a token containing username, permission, date, expiry date and digital signature 
        self.token = self.user.username + splitter + self.user.permission + splitter + str(self.date) + splitter + str(self.calcExpiryDate()) + splitter + self.system.systemName
        self.token = self.token + splitter + self.createDigitalSignature()
        return self.encryptToken(self.token)
        
    def createDigitalSignature(self):
        #digital signature uses the username and system name with added special characters
        signature = self.user.username +'!£$%^&*' + self.system.systemName
        #hash the signature 
        return self.hashString(signature)
    
    def hashString(self, plainString): 
        # convert string to array of bytes 
        bytes = plainString.encode('utf-8')
        #generate salt 
        salt = bcrypt.gensalt()
        #hash and salt string 
        hashString = bcrypt.hashpw(bytes, salt)
        #decode to prevent encoding twice 
        return hashString.decode('utf-8')
    
    def encryptToken(self, token):
        #encrypt the token using Encryption class 
        encrypted_token = self.encrypt.encrypt_data(token)
        return encrypted_token

In [88]:
class System:    
    #set default permission to 'u' (user) for creating a new account
    DEFAULT_PERMISSION = "u"
   
    def __init__(self, systemName):
        # replace spaces from system name 
        self.systemName = systemName.replace(' ', '_')
        # create instance of encryption class 
        self.encrypt = Encryption(self)
        
    #generate a key for the system     
    def createSystemKey(self):
        self.encrypt.create_new_keys()
        
    def login(self, username, password): 
        with open(self.systemName + 'users.csv', newline='') as userscsv:
            fieldnames = ['username', 'password', 'permission']
            reader = csv.DictReader(userscsv, fieldnames=fieldnames)
            for row in reader:
                if row['username'] == username:
                    #convert password to bytes 
                    enteredPassword = password.encode('utf-8')
                    
                    savedPassword = row['password']
                
                    #check password 
                    if bcrypt.checkpw(enteredPassword, savedPassword.encode('utf-8')):
                        if row['permission'] == 'u':
                            return User(row['username'], row['password'], row['permission'])
                        elif row['permission'] == 'a':
                            return AdminUser(row['username'], row['password'], row['permission'])
            #return null if there is no match
            return None
    
            
    def hashpassword(self, plainPassword):
        #convert password to array of bytes 
        bytes = plainPassword.encode('utf-8')
        
        #generate salt 
        salt = bcrypt.gensalt()
        
        #hash and salt password 
        hashpassword = bcrypt.hashpw(bytes, salt)
        return hashpassword
        
        
    def loginWithToken(self, token):
        token = self.encrypt.decrypt_data(token)
        if token == None:
            return None
        else:
            #encode token to bytes for processing 
            token = token.encode('utf-8')
            user = self.checkTokenIsValid(token)
            return user
            
        
    def getPermFromToken(self, token):
        #retrieve permission from token 
        perm = token.split(bytes('!£$', 'utf-8'))[1]
        #return permission as a string 
        return perm.decode('utf-8')
        
        
    def checkTokenIsValid(self, token):
        #get username 
        username = self.getUserFromToken(token)
        #get permission of user 
        perm = self.getPermFromToken(token)
        #get system name 
        sys_name = token.split(bytes('!£$', 'utf-8'))[4]
        sys_name = sys_name.decode('utf-8')
        
        #check token is for the right system 
        if self.systemName == sys_name:
            #check username is stored within the CSV file 
            with open(self.systemName + 'users.csv', newline='') as userscsv:
                fieldnames = ['username', 'password', 'permission']
                reader = csv.DictReader(userscsv, fieldnames=fieldnames)
                for row in reader:
                    #check the username and permission match what's stored 
                    if row['username'] == username and row['permission'] == perm:
                        #retrieve the expiry date 
                        expiry_str = token.split(bytes('!£$', 'utf-8'))[3]
                        expiry_str = expiry_str.decode('utf-8')
                        #convert expiry date to a datetime object to compare to today's date 
                        expiry_date = datetime.strptime(expiry_str, '%Y-%m-%d').date()
                        #check expiry date 
                        if date.today() < expiry_date:
                            #check signature 
                            if self.checkSignature(token):
                                #check whether user is an admin 
                                if row['permission'] == 'u':
                                    return User(row['username'], row['password'], row['permission'])
                                elif row['permission'] == 'a':
                                    return AdminUser(row['username'], row['password'], row['permission'])
        #return null if there is no match
        return None
    
    def checkSignature(self, token):
        hashed_sig = token.split(bytes('!£$', 'utf-8'))[5]
        hashed_sig = hashed_sig.decode('utf-8')
        plain_sig = self.getUserFromToken(token) + '!£$%^&*' + self.systemName
        #check signature 
        if bcrypt.checkpw(plain_sig.encode('utf-8'), hashed_sig.encode('utf-8')):
            return True
        else:
            return False
    
    def getUserFromToken(self, token): 
        #retrieve username from the token 
        username = token.split(bytes('!£$', 'utf-8'))[0]
        #return username as a string 
        return username.decode('utf-8')
    
    
    def createAccount(self, user):
        with open(self.systemName + 'users.csv', 'a', newline='') as userscsv:
            fieldnames = ['username', 'password', 'permission', 'token']
            writer = csv.DictWriter(userscsv, delimiter=',', quotechar='|', 
                                   quoting=csv.QUOTE_MINIMAL, fieldnames=fieldnames)
            hashedpw = self.hashpassword(user.password)
        
            writer.writerow({'username': user.username, 'password': hashedpw.decode('utf-8'), 'permission': user.permission})

    def read_csv(self):
        with open(self.systemName + 'users.csv', newline='') as userscsv:
            fieldnames = ['username', 'password', 'permission']
            reader = csv.DictReader(userscsv, fieldnames=fieldnames)
            for row in reader:
                print("", row['username'], row['password'], row['permission'])
            
    def display_main_menu(self):
        while 1: 
            print(self.systemName)
            print("___________________")
            print("Main Menu")
            print("1. Login")
            print("2. Create Account")
            print("3. Login With Access Token")
            print("0. Exit")
            
            try: 
                user_command = int(input("Enter your command: "))
            except ValueError: 
                #set user command to minus one to print invalid command message 
                user_command = -1
            
            #if else statement to carry out user command 
            if user_command == 1:
                username = input("Enter Username: ")
                #mask password for security 
                password = maskpass.advpass()
                user = self.login(username,  password)
                if user is None:
                    print("Incorrect username or password")                    
                else:
                    user.userAccountPage(self)
            elif user_command == 2:
                username = input("Enter new Username: ")
                if username != "":
                    #mask the password for security 
                    password = maskpass.advpass()
                    if password != "":
                        self.createAccount(User(username, password, self.DEFAULT_PERMISSION))
                        print("New Account Created for: ", username)
                    else:
                        print("No password provided")
                else: 
                    print("No username provided")
            elif user_command == 3:
                accessToken = input("Enter Access Token: ")
                user = self.loginWithToken(accessToken)
                if user is None:
                    print("Invalid token supplied")
                else: 
                    print("Valid Access Token Supplied. Logging in")
                    #give time for user to see acceptance message 
                    time.sleep(2)
                    user.userAccountPage(self)
            elif user_command == 0:
                print("Session Ended")
                break
            else: 
                clear_output(wait=True)
                print("Invalid Command")    

In [89]:
class Encryption: 
    
    def __init__(self, system):
        self.system = system
        #filepath to save private key 
        self.privateKeyPath = "privateKey-" + self.system.systemName + ".txt"
        #filepath to save encrypted symmetric key 
        self.symmetricKeyPath = "symmetricKey-" + self.system.systemName + ".txt"
        
    def create_new_keys(self):
        #generate public & private keys 
        publicKey, privateKey = rsa.newkeys(2048)
        #generate symmetric key 
        symmetricKey = Fernet.generate_key()
        #encrypt the symmetric key 
        enc_symmetricKey = self.encrypt_symmetric_key(symmetricKey, publicKey)
        #save keys to files 
        self.save_keys(enc_symmetricKey, privateKey)
        
    def encrypt_symmetric_key(self, symmetricKey, publicKey):
        #encrypt symmetric key 
        enc_symmetricKey = rsa.encrypt(symmetricKey, publicKey)
        #convert to base64 to save to file 
        b64_enc_symmetricKey = b64encode(enc_symmetricKey).decode('utf-8')
        symmetricKey = b64_enc_symmetricKey
        print("Symmetric Key Encryption successful")
        return symmetricKey
    
    def save_keys(self, symmetricKey, privateKey):
        self.write_file(self.symmetricKeyPath, symmetricKey)
        file = open(self.privateKeyPath, 'wb')
        file.write(privateKey.save_pkcs1('PEM'))
        print("Keys saved")
        
    def encrypt_data(self, data):
        key = self.getSymmetricKey()
        #check key has been retrieved 
        if key != None: 
            fernet = Fernet(key)
            enc_data = fernet.encrypt(data.encode('utf-8'))
            return enc_data  
        else:
            #failed to retrieve key 
            return None
        
    def getPrivateKey(self):
        try: 
            with open(self.privateKeyPath, 'rb') as file:
                #load the private key as a RSA.PrivateKey object
                privateKey = rsa.PrivateKey.load_pkcs1(file.read(), format='PEM')
            return privateKey
        except:
            return None
            
    def getSymmetricKey(self):
        #get symmetric key from file 
        enc_symmetricKey = self.read_file(self.symmetricKeyPath)
        #make sure file read has been successful 
        if enc_symmetricKey != None:
            enc_symmetricKey = b64decode(enc_symmetricKey)
            #decrypt symmetric key 
            symmetricKey = rsa.decrypt(enc_symmetricKey, self.getPrivateKey())
            return symmetricKey
        else: 
            #file read has been unsuccessful 
            return None
        
    def decrypt_data(self, enc_data):
        key = self.getSymmetricKey()
        #check key has been retrieved 
        if key != None:
            fernet = Fernet(key)
            #decrypt data using the symmetric key 
            plain_data = fernet.decrypt(enc_data).decode('utf-8')
            return plain_data
        else:
            #failed to retrieve key 
            return None
    
    #method to write to a file 
    def write_file(self, path, text): 
        file = open(path, 'w')
        try:
            return file.write(text.decode('utf-8'))
        except:
            return file.write(text)
        
    #method to read from a file 
    def read_file(self, path):
        try: 
            file = open(path, 'r')
            return file.read()
        except:
            return None
        

In [90]:
system = System("Super Secure System")

system.display_main_menu()

system1 = System("Another System")

system1.display_main_menu()

Super_Secure_System
___________________
Main Menu
1. Login
2. Create Account
3. Login With Access Token
0. Exit
Enter your command: 0
Session Ended
Another_System
___________________
Main Menu
1. Login
2. Create Account
3. Login With Access Token
0. Exit
Enter your command: 3
Enter Access Token: token
Invalid token supplied
Another_System
___________________
Main Menu
1. Login
2. Create Account
3. Login With Access Token
0. Exit
Enter your command: 0
Session Ended
